# ShohnomaLLM - Обучение модели

Fine-tuning Qwen2.5-1.5B для генерации таджикских стихов.

**Требования:**
- GPU: T4 (15GB VRAM) или лучше
- Данные в репозитории (клонируется автоматически)

**Время обучения:** ~2-3 часа на T4, ~1 час на L4

In [ ]:
# Проверка GPU
!nvidia-smi

In [ ]:
# Установка зависимостей
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# Клонируем репозиторий ShohnomaLLM
!git clone https://github.com/Kuchizu/ShohnomaLLM.git
%cd ShohnomaLLM
!pip install -r requirements.txt -q

In [ ]:
# Подключение Google Drive (для сохранения модели)
from google.colab import drive
drive.mount('/content/drive')

# Пути
REPO_DIR = "/content/ShohnomaLLM"
DATA_DIR = f"{REPO_DIR}/data"  # Данные из репозитория
MODEL_DIR = "/content/drive/MyDrive/ShohnomaLLM/models"  # Модели в Drive

# Создаём директории
!mkdir -p {DATA_DIR}/processed {DATA_DIR}/training {MODEL_DIR}

## 1. Подготовка данных

Используем модули из репозитория

In [ ]:
import sys
sys.path.insert(0, REPO_DIR)

from training.format_dataset import DatasetFormatter
from training.config import get_config

# Загружаем конфигурацию
config = get_config("colab_t4")  # или "colab_a100" для A100
print(f"Модель: {config.model.base_model}")
print(f"LoRA rank: {config.lora.r}")

In [ ]:
# Форматируем датасет
formatter = DatasetFormatter()

# Путь к сырым данным (из склонированного репозитория)
raw_data = f"{DATA_DIR}/raw/ganjoor/all_classical.jsonl"

# Проверяем наличие данных
import os
if os.path.exists(raw_data):
    print(f"Данные найдены: {raw_data}")
    
    # Форматируем
    processed_file = f"{DATA_DIR}/processed/classical.jsonl"
    
    count = formatter.process_jsonl(raw_data, processed_file, source_type="ganjoor")
    print(f"Обработано: {count} примеров")
    
    # Разбиваем на train/val
    formatter.create_train_val_split(processed_file, f"{DATA_DIR}/training")
else:
    print(f"ОШИБКА: Данные не найдены по пути: {raw_data}")
    print(f"Проверьте, что файл data/raw/ganjoor/all_classical.jsonl есть в репозитории")

## 2. Загрузка модели

In [ ]:
from unsloth import FastLanguageModel
import torch

# Загрузка модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config.model.base_model,
    max_seq_length=config.model.max_seq_length,
    dtype=None,
    load_in_4bit=config.model.load_in_4bit,
)

print(f"Модель загружена: {config.model.base_model}")

In [ ]:
# Добавляем LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=config.lora.r,
    target_modules=config.lora.target_modules,
    lora_alpha=config.lora.lora_alpha,
    lora_dropout=config.lora.lora_dropout,
    bias=config.lora.bias,
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("LoRA добавлен")
model.print_trainable_parameters()

## 3. Обучение

In [ ]:
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# Загрузка датасета
dataset = load_dataset(
    'json',
    data_files={
        'train': f"{DATA_DIR}/training/train.jsonl",
        'validation': f"{DATA_DIR}/training/val.jsonl",
    }
)

print(f"Train: {len(dataset['train'])}")
print(f"Val: {len(dataset['validation'])}")

In [ ]:
# Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    dataset_text_field="text",
    max_seq_length=config.model.max_seq_length,
    args=TrainingArguments(
        output_dir="./outputs",
        per_device_train_batch_size=config.training.per_device_train_batch_size,
        per_device_eval_batch_size=config.training.per_device_eval_batch_size,
        gradient_accumulation_steps=config.training.gradient_accumulation_steps,
        learning_rate=config.training.learning_rate,
        lr_scheduler_type=config.training.lr_scheduler_type,
        warmup_ratio=config.training.warmup_ratio,
        num_train_epochs=config.training.num_train_epochs,
        bf16=config.training.bf16,
        optim=config.training.optim,
        weight_decay=config.training.weight_decay,
        max_grad_norm=config.training.max_grad_norm,
        logging_steps=config.training.logging_steps,
        eval_steps=config.training.eval_steps,
        eval_strategy=config.training.evaluation_strategy,
        save_steps=config.training.save_steps,
        save_strategy=config.training.save_strategy,
        save_total_limit=config.training.save_total_limit,
        load_best_model_at_end=config.training.load_best_model_at_end,
        seed=config.training.seed,
        report_to=config.training.report_to,
    ),
)

In [ ]:
# Запуск обучения
print("Начало обучения...")
trainer.train()

## 4. Сохранение модели

In [ ]:
# Сохраняем LoRA
lora_path = f"{MODEL_DIR}/tajik-poetry-lora"
model.save_pretrained(lora_path)
tokenizer.save_pretrained(lora_path)
print(f"LoRA сохранён: {lora_path}")

In [ ]:
# Объединяем LoRA с базовой моделью (16-bit)
merged_path = f"{MODEL_DIR}/tajik-poetry-1.5b"

model.save_pretrained_merged(
    merged_path,
    tokenizer,
    save_method="merged_16bit",
)
print(f"Merged модель: {merged_path}")

## 5. Тестирование

In [ ]:
# Режим inference
FastLanguageModel.for_inference(model)

# Системный промпт
from training.format_dataset import SYSTEM_PROMPT

def generate_poem(prompt, max_tokens=256, temperature=0.8):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True,
    )
    
    return tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True,
    )

In [ ]:
# Тестовые промпты
test_prompts = [
    "Рубоӣ бинавис",
    "Ғазали ошиқона эҷод кун",
    "Шеър дар бораи баҳор бинавис",
]

for prompt in test_prompts:
    print(f"\n{'='*50}")
    print(f"Запрос: {prompt}")
    print(f"{'='*50}")
    print(generate_poem(prompt))

## 6. Экспорт в GGUF (опционально)

Для запуска на CPU через llama.cpp

In [ ]:
# Экспорт в GGUF
gguf_path = f"{MODEL_DIR}/tajik-poetry-q4"

model.save_pretrained_gguf(
    gguf_path,
    tokenizer,
    quantization_method="q4_k_m",
)
print(f"GGUF сохранён: {gguf_path}")

In [ ]:
print("\n" + "="*50)
print("Обучение завершено!")
print("="*50)
print(f"\nМодели сохранены в Google Drive: {MODEL_DIR}")
print("\nСкачайте модель и используйте локально:")
print("  python -m cli.generate --model путь/к/модели")